In [1]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 97 kB 6.7 MB/s 
     |████████████████████████████████| 140 kB 51.5 MB/s 
     |████████████████████████████████| 180 kB 63.1 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=28932cf43b350c5929f21637d7cc11263374b0c3c294c99bfe6be00460479590
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=da191b3022c7876764803cd209a994b8e3b20c6be9451ebbf0678ad8d22e48d6
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [ ]:
__PROJECT_SOURCE__="COLAB"

In [2]:
if __PROJECT_SOURCE__=="COLAB":
    # Import PyDrive and associated libraries.
    # This only needs to be done once per notebook.
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    # Authenticate and create the PyDrive client.
    # This only needs to be done once per notebook.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    from google.colab import drive
    drive.mount('/content/drive')
    fname_dir="/content/blood/"
    fname_url="/content/drive/MyDrive/blood/rework/*agg.csv"
    fname=fname_url.split("/")[-1]
elif __PROJECT_SOURCE__=="LOCAL":
    fname_dir="sdfsdfsd" #working dir
    fname_url="/content/drive/MyDrive/blood/rework/*agg.csv" #data source dir
    


Mounted at /content/drive


In [6]:
!mkdir $fname_dir

In [7]:
!cp $fname_url $fname_dir

In [8]:
if __PROJECT_SOURCE__=="COLAB":
    drive.flush_and_unmount()
    print('Unmount Google Drive :-(')

Unmount Google Drive :-(


## Feldolgozás

In [3]:
!rm *.hdf5

rm: cannot remove '*.hdf5': No such file or directory


In [9]:
import pandas as pd

In [10]:
fnames_list=["ldl1_agg.csv","ldl2_agg.csv"]

In [12]:
file_name1=fname_dir+fnames_list[0]
print(file_name1)

/content/blood/ldl1_agg.csv


In [13]:
file_name2=fname_dir+fnames_list[1]
print(file_name2)

/content/blood/ldl2_agg.csv


In [16]:
df_agg1= pd.read_csv(file_name1)
df_agg1.describe()
df_agg1.drop(df_agg1[df_agg1.absorbance0 < 0].index, inplace=True) # kill the negative elements


In [17]:
df_agg2= pd.read_csv(file_name2)
df_agg2.describe()
df_agg2.drop(df_agg2[df_agg2.absorbance0 < 0].index, inplace=True) # kill the negative elements


In [18]:
df_agg=pd.concat([df_agg1,df_agg2], ignore_index=True)

In [19]:
df_agg.tail()

,Unnamed: 0,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,donation_id,temperature,humidity,cholesterol_ldl_value
537,405,0.559508,0.557981,0.562073,0.566272,0.571244,0.577397,0.583213,0.590163,0.598694,0.610475,0.623472,0.636641,0.642579,0.645626,0.647103,0.645668,0.643687,0.640753,0.637975,0.635578,0.633089,0.630470,0.627600,0.624106,0.622028,0.619228,0.617372,0.616006,0.614953,0.614089,0.614401,0.614494,0.615416,0.617037,0.619308,0.623084,0.626258,0.629867,0.634516,...,1.451427,1.440180,1.429945,1.421697,1.412292,1.403772,1.395936,1.385628,1.377189,1.370285,1.363224,1.354198,1.347067,1.339531,1.332175,1.326958,1.320780,1.315768,1.311262,1.304993,1.299778,1.296999,1.292156,1.289398,1.285569,1.286972,1.283523,1.281386,1.279686,1.274868,1.271164,1.265842,1.263005,1.266337,1.297181,1.266201,11886,36.536500,65.203000,88.54285
538,406,0.587427,0.585250,0.589764,0.594061,0.599806,0.604902,0.609408,0.614733,0.620852,0.630668,0.641564,0.653106,0.657908,0.659866,0.660317,0.658655,0.655788,0.652237,0.648927,0.646154,0.643198,0.640632,0.637537,0.633689,0.631356,0.628085,0.625785,0.624119,0.622667,0.621206,0.620691,0.620383,0.620737,0.621599,0.623025,0.625937,0.628400,0.631338,0.635659,...,1.475231,1.463203,1.454463,1.445494,1.436059,1.426811,1.417521,1.406092,1.397415,1.388936,1.383776,1.372381,1.364249,1.355797,1.347775,1.341057,1.333965,1.328754,1.321832,1.315572,1.309635,1.303772,1.299180,1.295138,1.292522,1.289935,1.287213,1.282831,1.277070,1.270260,1.253691,1.242504,1.236326,1.236166,1.250518,1.236380,11889,39.054667,56.012500,159.68645
539,407,0.477241,0.481792,0.483792,0.489861,0.497037,0.504735,0.509637,0.514202,0.520144,0.530000,0.541345,0.550381,0.555803,0.558563,0.559343,0.558502,0.556676,0.554130,0.551494,0.547095,0.543851,0.540543,0.537108,0.533737,0.530775,0.527208,0.524530,0.523219,0.521271,0.520013,0.519327,0.518718,0.518963,0.519878,0.520868,0.522727,0.525264,0.528919,0.532656,...,1.513493,1.494378,1.478303,1.463623,1.448563,1.434307,1.419192,1.402043,1.388613,1.375551,1.366222,1.351601,1.340437,1.327134,1.318319,1.309765,1.300136,1.292696,1.283508,1.275061,1.269340,1.263197,1.258333,1.253829,1.251677,1.249435,1.246182,1.243464,1.237783,1.227517,1.206936,1.179302,1.162252,1.156460,1.167845,1.159627,11890,46.116000,46.849000,99.20000
540,408,0.565670,0.564820,0.569749,0.573770,0.580790,0.586531,0.591289,0.596792,0.603492,0.613606,0.625469,0.637730,0.642756,0.645300,0.646095,0.644386,0.641853,0.638425,0.635619,0.633162,0.630523,0.627955,0.624878,0.621162,0.618793,0.615519,0.613485,0.611918,0.610617,0.609168,0.608868,0.608688,0.609372,0.610378,0.612109,0.615408,0.618294,0.621513,0.626012,...,1.520369,1.506006,1.496877,1.486883,1.475934,1.466819,1.456462,1.445260,1.435506,1.426113,1.419729,1.409409,1.402097,1.391310,1.384826,1.377594,1.371748,1.366276,1.358685,1.354177,1.348492,1.342599,1.337098,1.332178,1.329692,1.329856,1.327782,1.323786,1.317777,1.305306,1.290804,1.276198,1.268350,1.263184,1.284706,1.269695,11891,40.384833,51.917167,164.71290
541,409,0.462006,0.463919,0.466006,0.4

In [20]:
df_agg.describe()

,Unnamed: 0,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,donation_id,temperature,humidity,cholesterol_ldl_value
count,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,...,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000
mean,170.702952,0.520451,0.522427,0.525972,0.530680,0.536854,0.543551,0.549648,0.555232,0.563119,0.574407,0.587313,0.598591,0.605160,0.608182,0.609431,0.609150,0.607543,0.605090,0.602129,0.598775,0.595773,0.592707,0.589487,0.585683,0.582415,0.579220,0.576781,0.574689,0.572943,0.571480,0.570644,0.570247,0.570480,0.571400,0.572933,0.575169,0.577869,0.581283,0.585061,...,1.510277,1.497814,1.485553,1.474109,1.463112,1.450626,1.439371,1.428135,1.417230,1.406798,1.398556,1.385953,1.377019,1.368533,1.360171,1.351819,1.344645,1.337181,1.330049,1.323455,1.316835,1.311906,1.307739,1.304195,1.301982,1.301178,1.300802,1.300394,1.299197,1.295539,1.289453,1.282270,1.275940,1.272953,1.287608,1.275556,6927.533210,42.111418,39.222955,111.436592
std,120.507914,0.040351,0.039984,0.040093,0.040035,0.040144,0.040521,0.040435,0.040376,0.040171,0.039981,0.039891,0.039888,0.039833,0.039825,0.039900,0.039866,0.039767,0.039718,0.039647,0.039748,0.039703,0.039702,0.039726,0.039846,0.040008,0.040042,0.040114,0.040156,0.040191,0.040191,0.040187,0.040152,0.040178,0.040161,0.040140,0.040145,0.040169,0.040266,0.040271,...,0.093496,0.091700,0.090305,0.088924,0.087603,0.085971,0.084668,0.083410,0.082486,0.081525,0.080697,0.079354,0.078591,0.077906,0.077287,0.076565,0.076015,0.075402,0.074832,0.074297,0.073843,0.073528,0.073310,0.073180,0.073283,0.073591,0.074264,0.075230,0.076718,0.079323,0.083310,0.088549,0.093928,0.098151,0.104133,0.101605,3094.955454,3.420590,8.948404,44.306810
min,0.000000,0.400606,0.403625,0.407266,0.411518,0.416126,0.421518,0.426629,0.433139,0.444069,0.456182,0.469249,0.480239,0.486473,0.489661,0.490777,0.490226,0.488814,0.486476,0.483668,0.479653,0.476146,0.473155,0.469834,0.466424,0.462388,0.459335,0.457297,0.455538,0.453853,0.452486,0.451719,0.451624,0.452026,0.453133,0.454851,0.457279,0.460600,0.463820,0.467916,...,1.212829,1.202957,1.193580,1.184336,1.175013,1.163368,1.154944,1.145616,1.137301,1.128859,1.118666,1.110864,1.102883,1.096296,1.089810,1.080864,1.075002,1.068992,1.063488,1.057937,1.052125,1.047685,

In [21]:

import tensorflow as tf
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout,Embedding, Flatten,LayerNormalization
from keras.models import Sequential, Model

import keras.optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld,mse

from keras.preprocessing.sequence import pad_sequences
import numpy as np


In [22]:
X_columns=df_agg.columns[1:-4]
y_columns=df_agg.columns[-1]
X_=df_agg[X_columns]
y_=df_agg[y_columns]

In [23]:
y_=np.array(y_)

In [24]:
y_=y_.reshape(-1,1)

In [25]:
y_

array([[105.6    ],
       [ 72.4    ],
       [ 68.4    ],
       [106.     ],
       [ 31.2    ],
       [ 40.6    ],
       [ 82.6    ],
       [ 32.7    ],
       [ 64.2    ],
       [151.     ],
       [ 94.8    ],
       [ 78.6    ],
       [110.8    ],
       [148.2    ],
       [ 82.6    ],
       [130.2    ],
       [211.6    ],
       [ 52.2    ],
       [ 91.2    ],
       [ 37.6    ],
       [121.6    ],
       [114.4    ],
       [114.6    ],
       [118.2    ],
       [ 78.2    ],
       [102.8    ],
       [188.4    ],
       [137.8    ],
       [134.     ],
       [165.4    ],
       [ 95.2    ],
       [ 68.2    ],
       [157.6    ],
       [157.6    ],
       [ 74.     ],
       [ 79.2    ],
       [147.4    ],
       [119.     ],
       [152.8    ],
       [127.04   ],
       [ 98.     ],
       [ 44.96   ],
       [ 63.18   ],
       [106.62   ],
       [ 85.58   ],
       [110.08   ],
       [ 86.56   ],
       [ 91.54   ],
       [122.28   ],
       [ 47.4    ],


In [26]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer


__SCALER__="minmax"

if __SCALER__=="normal":
    X_normalizer = Normalizer() 
    X=X_normalizer.fit_transform(X_)
    y_normalizer = Normalizer() 
    y=y_normalizer.fit_transform(y_)


elif __SCALER__=="minmax":
    X_minmax = MinMaxScaler() 
    X=X_minmax.fit_transform(X_)
    y_minmax = MinMaxScaler() 
    
    y=y_minmax.fit_transform(y_)
    




In [27]:
X

array([[0.45442072, 0.45946018, 0.46352764, ..., 0.82316512, 0.78118723,
        0.83334109],
       [0.46412324, 0.46056607, 0.46502857, ..., 0.60919169, 0.58001034,
        0.57221906],
       [0.2704717 , 0.27493718, 0.28373915, ..., 0.54688144, 0.51864758,
        0.55100822],
       ...,
       [0.25883836, 0.26735063, 0.26338547, ..., 0.30637068, 0.31488477,
        0.33420908],
       [0.55751512, 0.55132694, 0.55922789, ..., 0.49308185, 0.50032984,
        0.51049083],
       [0.20738232, 0.20622139, 0.20216907, ..., 0.31621416, 0.3301991 ,
        0.34781044]])

In [109]:
import wandb
from wandb.keras import WandbCallback

wandb.init(config={"hyper": "parameter"})




In [110]:
def blood_model_2():
    #clear_session()

    keras.backend.clear_session()

    input_len=170

    print(input_len)
    output_size=24
    drop_frac0=0.0 
    drop_frac1=0.0



    input1=Input(shape=(input_len,))

    #flatt=Flatten()(lstm1)

    non=42
    #initializer = tf.keras.initializers.LecunNormal()
    #initializer=tf.keras.initializers.LecunUniform()
    #initializer=tf.keras.initializers.HeUniform(    seed=None)
    #initializer= tf.keras.initializers.RandomNormal(    mean=3.0, stddev=0.05, seed=None)

    initializer="glorot_uniform"
    d1=Dense(5000,activation="relu",kernel_initializer=initializer)(input1)
    d1=Dense(500,activation="relu",kernel_initializer=initializer)(d1)
    d1=Dense(50,activation="relu",kernel_initializer=initializer)(d1)



    pred=Dense(1,)(d1)

    
    
    model = Model(inputs=input1, outputs=pred)

    opt = tf.keras.optimizers.Adamax(learning_rate=0.001)


    #lossfn = tf.keras.metrics.SparseCategoricalCrossentropy()

    model.compile(loss="mse",
        optimizer=opt,
        metrics=[None])
    return(model)

In [116]:
model_name="_col"
def scheduler(epoch, lr):
    return 0.00001

callback_LR = tf.keras.callbacks.LearningRateScheduler(scheduler)

callbacks = [callback_LR,
        
        #savemodela,
        ModelCheckpoint(filepath=model_name+"_{loss:.5f}_{val_loss:.5f}_.hdf5", monitor='val_loss',
                        verbose=1, save_best_only=True, mode='min')]

In [112]:
new_model_2=blood_model_2()

170


In [113]:
new_model_2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 170)]             0         
                                                                 
 dense (Dense)               (None, 5000)              855000    
                                                                 
 dense_1 (Dense)             (None, 500)               2500500   
                                                                 
 dense_2 (Dense)             (None, 50)                25050     
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 3,380,601
Trainable params: 3,380,601
Non-trainable params: 0
_________________________________________________________________


In [114]:
!rm *.hdf5

rm: cannot remove '*.hdf5': No such file or directory


In [35]:
#new_model_2.load_weights("./_col_0.00098_0.00173_.xhdf5")

In [117]:
history=new_model_2.fit(X,
          y,
          epochs=500, 
          batch_size=3,
          validation_split=0.2,
          verbose=1,
          callbacks=[callbacks,WandbCallback()],
          shuffle=True
          
          )

Epoch 1/500
145/145 [==============================] - ETA: 0s - loss: 0.0155
Epoch 00001: val_loss improved from inf to 0.03046, saving model to _col_0.01554_0.03046_.hdf5
145/145 [==============================] - 6s 42ms/step - loss: 0.0155 - val_loss: 0.0305 - lr: 1.0000e-05
Epoch 2/500
145/145 [==============================] - ETA: 0s - loss: 0.0152
Epoch 00002: val_loss did not improve from 0.03046
145/145 [==============================] - 6s 39ms/step - loss: 0.0152 - val_loss: 0.0307 - lr: 1.0000e-05
Epoch 3/500
145/145 [==============================] - ETA: 0s - loss: 0.0151
Epoch 00003: val_loss improved from 0.03046 to 0.02985, saving model to _col_0.01509_0.02985_.hdf5
145/145 [==============================] - 6s 41ms/step - loss: 0.0151 - val_loss: 0.0299 - lr: 1.0000e-05
Epoch 4/500
144/145 [============================>.] - ETA: 0s - loss: 0.0149
Epoch 00004: val_loss did not improve from 0.02985
145/145 [==============================] - 6s 40ms/step - loss: 0.0150 

KeyboardInterrupt: ignored

In [50]:
y_pred=new_model_2.predict(X)
result_df=pd.DataFrame()
y_pred_col=[i[0] for i in y_pred]
y_col=[i[0] for i in y]

result_df["y"]=y_col
result_df["y_pred"]=y_pred_col

In [51]:
import plotly.express as px

fig = px.scatter(result_df, x="y", y="y_pred",width=600, height=400 )
fig.show()